# Plagiarism Detection Model

Now that we've created training and test data, we are ready to define and train a model. Our goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features we provide the model.

This task will be broken down into a few discrete steps:

* Upload our data to S3.
* Define a binary classification model and a training script.
* Train our model and deploy it.
* Evaluate our deployed classifier.

## Load Data to S3

In the last notebook, we have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, we can upload our locally stored data to S3.

In [2]:
import pandas as pd
import boto3
import sagemaker
import os

In [3]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## Upload your training data to S3

Specify the `data_dir` where we saved our `train.csv` file. Decide on a descriptive `prefix` that defines where our data will be uploaded in the default S3 bucket. Finally, create a pointer to our training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data).

We shoul upload our entire directory. Later, the training script will only access the `train.csv` file.

In [4]:
# should be the name of directory created to save our features data
data_dir = './plagiarism_data'

# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/plagiarism_detection'

# upload all data to S3
test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)
train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

# Modeling

Now that we've uploaded our training data, it's time to define and train a model!

For a binary classification task, we can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

## Complete a training script 

To implement a custom classifier, we'll need `train.py` script (the script that is responsible for training our final model) given in the folder `source_sklearn` which holds code for a custom Scikit-learn model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model

1. Import any extra libraries we need
2. Define any additional model training hyperparameters using `parser.add_argument`
3. Define a model in the `if __name__ == '__main__':` section
4. Train the model in that same section

In [5]:
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

from sklearn.externals import joblib

## Import additional libraries needed to define a model
from sklearn.neighbors import KNeighborsClassifier

# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser()

    # SageMaker parameters, like the directories for training data and saving models; set automatically
    # Do not need to change
 

### Provided code

If you read the code above, you can see that the code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html)

# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function we specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **entry_point**: The path to the Python script SageMaker runs for training.
* **source_dir**: The path to the training script directory `train_sklearn` OR `train_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

## Define a Scikit-learn estimator

To import our desired estimator, use the following line:
```
from sagemaker.sklearn.estimator import SKLearn
```

In [6]:
# import and estimator code
from sagemaker.sklearn.estimator import SKLearn

train_script_path = './source_sklearn/train.py'

model = SKLearn(entry_point = train_script_path, framework_version='0.20.0', role = role, train_instance_type = 'ml.c4.xlarge')

## Train the estimator

Train our estimator on the training data stored in S3. This should create a training job that we can monitor in our SageMaker console.

In [7]:
%%time

# Train your estimator on S3 training data
model.fit({'train': train_location})


2020-07-02 17:21:05 Starting - Starting the training job...
2020-07-02 17:21:07 Starting - Launching requested ML instances......
2020-07-02 17:22:26 Starting - Preparing the instances for training......
2020-07-02 17:23:21 Downloading - Downloading input data...
2020-07-02 17:23:51 Training - Downloading the training image..2020-07-02 17:24:12,798 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-07-02 17:24:12,800 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-07-02 17:24:12,810 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-07-02 17:24:13,257 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setup.py
2020-07-02 17:24:13,258 sagemaker-containers INFO     Generating setup.cfg
2020-07-02 17:24:13,258 sagemaker-containers INFO     Generating MANIFEST.in
2020-07-02 17:24:13,258 sagemaker-containers INFO     Installing module with the following c

## Deploy the trained model

After training, deploy our model to create a `predictor`.

To deploy a trained model, we'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type.

In [8]:
%%time

# deploy model to create a predictor
predictor = model.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1)

-------------------------!CPU times: user 391 ms, sys: 15.7 ms, total: 407 ms
Wall time: 12min 33s


# Evaluating Model

Once our model is deployed, we can see how it performs when applied to test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [9]:
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## Determine the accuracy of the model

Use our deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. We can use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

In [10]:
# First: generate predicted, class labels
test_y_preds = predictor.predict(test_x)

# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [11]:
from sklearn.metrics import accuracy_score
# Second: calculate the test accuracy
accuracy = accuracy_score(test_y, test_y_preds)

print(accuracy)


## print out the array of predicted and true labels
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

0.96

Predicted class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


## Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`.

In [12]:
# deleting endpoint
predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [13]:
# deleting bucket

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': 'AC2621A7D6CF7794',
   'HostId': 'G3IeuBv/NABbMr/JK1epEvJw79axXCdaMkr4B4YMCwul+G8S5E6CTQzOS3AlKvnoVT6AZnjTyt0=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'G3IeuBv/NABbMr/JK1epEvJw79axXCdaMkr4B4YMCwul+G8S5E6CTQzOS3AlKvnoVT6AZnjTyt0=',
    'x-amz-request-id': 'AC2621A7D6CF7794',
    'date': 'Thu, 02 Jul 2020 17:37:23 GMT',
    'connection': 'close',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker-scikit-learn-2020-07-02-17-21-04-699/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker/plagiarism_detection/train.csv'},
   {'Key': 'sagemaker-pytorch-2020-06-12-20-16-31-481/output/model.tar.gz'},
   {'Key': 'sagemaker-pytorch-2020-06-12-19-42-58-928/source/sourcedir.tar.gz'},
   {'Key': 'sagemaker-pytorch-2020-06-12-19-42-58-928/debug-output/training_job_end.ts'},
   {'Key': 'sagemaker-pytorch-2020-06-12-20-34-39-044/source

## Further Directions

There are many ways to improve or add on to this project to expand our learning or make this more of a unique project. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.